# Pin study: comparing CE and multigroup results within OpenMC

## Import modules

In [1]:
import sys
import os

import openmc
import openmc.mgxs as mgxs

import numpy as np
import matplotlib.pyplot as plt
import copy as copy

import pandas as pd
from IPython.display import Image
import glob

## Add location of OpenMC XS executable, setup ENDF xs path

In [2]:
import os

# Add path to OpenMC binary
# os.environ['PATH'] += r':/path/to/openmc/bin'
os.environ['PATH'] += r':/Users/jean.ragusa/repo/openmc/local/bin'

# Add location of OpenMC xs data
%env OPENMC_CROSS_SECTIONS=/Users/jean.ragusa/repo/endfb-viii.0-hdf5/cross_sections.xml

env: OPENMC_CROSS_SECTIONS=/Users/jean.ragusa/repo/endfb-viii.0-hdf5/cross_sections.xml


## Cleanup .xml files 

In [3]:
clean_file_list = ["model.xml", "materials.xml", "geometry.xml","plots.xml","geometry.xml",\
                   "settings.xml","tallies.out","summary.h5","statepoint.50.h5"]

for file in clean_file_list :
    path_file = os.path.join(os.getcwd(), file)
    if os.path.exists(path_file):
        os.remove(path_file)

## Start model

In [4]:
model=openmc.Model()

### Define Materials

In [5]:
uo2 = openmc.Material(name='uo2')

uo2.add_nuclide('U234', 6.11864E-06, 'ao')
uo2.add_nuclide('U235', 7.18132E-04, 'ao')
uo2.add_nuclide('U236', 3.29861E-06, 'ao')
uo2.add_nuclide('U238', 2.21546E-02, 'ao')
uo2.add_nuclide('O16', 4.57642E-02, 'ao')

uo2.set_density('g/cm3', 10.257 )

In [6]:
zirconium = openmc.Material(name="zirconium")

zirconium.add_nuclide('Zr90', 2.18865E-02, 'ao')
zirconium.add_nuclide('Zr91', 4.77292E-03, 'ao')
zirconium.add_nuclide('Zr92', 7.29551E-03, 'ao')
zirconium.add_nuclide('Zr94', 7.39335E-03, 'ao')
zirconium.add_nuclide('Zr96', 1.19110E-03, 'ao')
zirconium.add_nuclide('Sn112',4.68066E-06, 'ao')
zirconium.add_nuclide('Sn114', 3.18478E-06, 'ao')
zirconium.add_nuclide('Sn115', 1.64064E-06, 'ao')
zirconium.add_nuclide('Sn116', 7.01616E-05, 'ao')
zirconium.add_nuclide('Sn117', 3.70592E-05, 'ao')
zirconium.add_nuclide('Sn118', 1.16872E-04, 'ao')
zirconium.add_nuclide('Sn119', 4.14504E-05, 'ao')
zirconium.add_nuclide('Sn120', 1.57212E-04, 'ao')
zirconium.add_nuclide('Sn122', 2.23417E-05, 'ao')
zirconium.add_nuclide('Sn124', 2.79392E-05, 'ao')
zirconium.add_nuclide('Fe54', 8.68307E-06, 'ao')
zirconium.add_nuclide('Fe56', 1.36306E-04, 'ao')
zirconium.add_nuclide('Fe57', 3.14789E-06, 'ao')
zirconium.add_nuclide('Fe58', 4.18926E-07, 'ao')
zirconium.add_nuclide('Cr50', 3.30121E-06, 'ao')
zirconium.add_nuclide('Cr52', 6.36606E-05, 'ao')
zirconium.add_nuclide('Cr53', 7.21860E-06, 'ao')
zirconium.add_nuclide('Cr54', 1.79686E-06, 'ao')
zirconium.add_nuclide('Hf174', 3.54138E-09, 'ao')
zirconium.add_nuclide('Hf176', 1.16423E-07, 'ao')
zirconium.add_nuclide('Hf177', 4.11686E-07, 'ao')
zirconium.add_nuclide('Hf178', 6.03806E-07, 'ao')
zirconium.add_nuclide('Hf179', 3.01460E-07, 'ao')
zirconium.add_nuclide('Hf180', 7.76449E-07, 'ao')

zirconium.set_density('g/cm3',  6.56)

In [7]:
water = openmc.Material(name="water")

water.add_nuclide('H1',4.96224E-02, 'ao')
water.add_nuclide('O16', 2.48112E-02, 'ao')
water.add_nuclide('B10', 1.07070E-05, 'ao')
water.add_nuclide('B11', 4.30971E-05, 'ao')

water.set_density('g/cm3', 0.743)

water.add_s_alpha_beta('c_H_in_H2O')

In [8]:
# materials_file = openmc.Materials([uo2, zirconium, water]) 

model.materials = openmc.Materials([uo2, zirconium, water]) 
print(model.materials)

[Material
	ID             =	1
	Name           =	uo2
	Temperature    =	None
	Density        =	10.257 [g/cm3]
	Volume         =	None [cm^3]
	Depletable     =	True
	S(a,b) Tables  
	Nuclides       
	U234           =	6.11864e-06  [ao]
	U235           =	0.000718132  [ao]
	U236           =	3.29861e-06  [ao]
	U238           =	0.0221546    [ao]
	O16            =	0.0457642    [ao]
, Material
	ID             =	2
	Name           =	zirconium
	Temperature    =	None
	Density        =	6.56 [g/cm3]
	Volume         =	None [cm^3]
	Depletable     =	False
	S(a,b) Tables  
	Nuclides       
	Zr90           =	0.0218865    [ao]
	Zr91           =	0.00477292   [ao]
	Zr92           =	0.00729551   [ao]
	Zr94           =	0.00739335   [ao]
	Zr96           =	0.0011911    [ao]
	Sn112          =	4.68066e-06  [ao]
	Sn114          =	3.18478e-06  [ao]
	Sn115          =	1.64064e-06  [ao]
	Sn116          =	7.01616e-05  [ao]
	Sn117          =	3.70592e-05  [ao]
	Sn118          =	0.000116872  [ao]
	Sn119          =	4.14504e-0

### Create pincell: fuel, clad, water (no gap)

In [9]:
pitch = 1.26
rod_height = 1.

fuel_outer_radius = openmc.ZCylinder(r=0.418)
clad_outer_radius = openmc.ZCylinder(r=0.475)

left   = openmc.XPlane(-pitch/2, boundary_type='reflective')
right  = openmc.XPlane( pitch/2, boundary_type='reflective')
bottom = openmc.YPlane(-pitch/2, boundary_type='reflective')
top    = openmc.YPlane( pitch/2, boundary_type='reflective')

In [10]:
fuel_region  = -fuel_outer_radius
clad_region  = +fuel_outer_radius & -clad_outer_radius
water_region = +left & -right & +bottom & -top & +clad_outer_radius

In [11]:
fuel = openmc.Cell(name='fuel')
fuel.fill = uo2
fuel.region = fuel_region
fuel.temperature = 565

In [12]:
clad = openmc.Cell(name='clad')
clad.fill = zirconium
clad.region = clad_region
clad.temperature = 565

In [13]:
moderator = openmc.Cell(name='moderator')
moderator.fill = water
moderator.region = water_region 
moderator.temperature = 565

In [14]:
root_universe = openmc.Universe(cells=(fuel, clad, moderator))
model.geometry = openmc.Geometry(root_universe)

### Plot

model.materials.export_to_xml()
model.geometry.export_to_xml()

plots = []

plot = openmc.Plot.from_geometry(model.geometry)
plot.pixels = (500, 500)
dr = pitch * 0.05
plot.width = (pitch+dr, pitch+dr)
plot.origin = (0., 0., 0)
plot.color_by = 'cell'

plots.append(plot)
model.plots=openmc.Plots(plots)
plot.to_ipython_image(openmc_exec='openmc', cwd='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/vera')

###  Settings

In [28]:
settings = openmc.Settings()

bounds = [-pitch/2, -pitch/2, -rod_height/2, pitch/2, +pitch/2, rod_height/2]
uniform_dist = openmc.stats.Box(bounds[:3], bounds[3:], only_fissionable=True)

source = openmc.IndependentSource(space=uniform_dist)

settings.source = source

settings.batches = 30
settings.inactive = 20
settings.particles = 20000
settings.keff_trigger = {'type':'std_dev','threshold':0.00030}
settings.trigger_active = True
settings.trigger_max_batches = 50000
settings.output = {'tallies':True}

settings.temperature['method'] = 'interpolation'

model.settings = settings

settings_file = settings

### Define and choose energy group

In [16]:
egroup_name = 'XMAS-172'
egroup = openmc.mgxs.GROUP_STRUCTURES[egroup_name]

### Create MGXS tallies for mgxs directory

In [17]:
mgxs_lib_order0 = openmc.mgxs.Library(model.geometry)
groups = openmc.mgxs.EnergyGroups(egroup)

mgxs_lib_order0.energy_groups = groups
mgxs_lib_order0.scatter_format = "legendre"
mgxs_lib_order0.mgxs_types = ['transport','nu-transport','absorption', 'nu-fission', 'fission' ,\
                              'chi', 'consistent scatter matrix', 'multiplicity matrix','total',\
                              'consistent nu-scatter matrix']

mgxs_lib_order0.legendre_order = 0

mgxs_lib_order0.correction = None
mgxs_lib_order0.by_nuclide = False
mgxs_lib_order0.domain_type = 'cell'
mgxs_lib_order0.domains = model.geometry.get_all_material_cells().values()
mgxs_lib_order0.build_library()
mgxs_lib_order0.check_library_for_openmc_mgxs()

In [18]:
mgxs_lib_order1 = openmc.mgxs.Library(model.geometry)
groups = openmc.mgxs.EnergyGroups(egroup)

mgxs_lib_order1.energy_groups = groups
mgxs_lib_order1.scatter_format = "legendre"
mgxs_lib_order1.mgxs_types = ['transport','nu-transport','absorption', 'nu-fission', 'fission' ,\
                              'chi', 'consistent scatter matrix', 'multiplicity matrix','total',\
                              'consistent nu-scatter matrix']

mgxs_lib_order1.legendre_order = 1

mgxs_lib_order1.by_nuclide = False
mgxs_lib_order1.domain_type = 'cell'
mgxs_lib_order1.domains = model.geometry.get_all_material_cells().values()
mgxs_lib_order1.build_library()
mgxs_lib_order1.check_library_for_openmc_mgxs()

/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 1 is greater than zero
  warn(msg, RuntimeWarning)


In [19]:
mgxs_lib_order3 = openmc.mgxs.Library(model.geometry)
groups = openmc.mgxs.EnergyGroups(egroup)

mgxs_lib_order3.energy_groups = groups
mgxs_lib_order3.scatter_format = "legendre"
mgxs_lib_order3.mgxs_types = ['transport','nu-transport','absorption', 'nu-fission', 'fission' ,\
                              'chi', 'consistent scatter matrix', 'multiplicity matrix','total',\
                              'consistent nu-scatter matrix']

mgxs_lib_order3.legendre_order = 3

mgxs_lib_order3.by_nuclide = False
mgxs_lib_order3.domain_type = 'cell'
mgxs_lib_order3.domains = model.geometry.get_all_material_cells().values()
mgxs_lib_order3.build_library()
mgxs_lib_order3.check_library_for_openmc_mgxs()

/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 3 is greater than zero
  warn(msg, RuntimeWarning)


In [20]:
mgxs_lib_order5 = openmc.mgxs.Library(model.geometry)
groups = openmc.mgxs.EnergyGroups(egroup)

mgxs_lib_order5.energy_groups = groups
mgxs_lib_order5.scatter_format = "legendre"
mgxs_lib_order5.mgxs_types = ['transport','nu-transport','absorption', 'nu-fission', 'fission' ,\
                              'chi', 'consistent scatter matrix', 'multiplicity matrix','total',\
                              'consistent nu-scatter matrix']

mgxs_lib_order5.legendre_order = 5

mgxs_lib_order5.by_nuclide = False
mgxs_lib_order5.domain_type = 'cell'
mgxs_lib_order5.domains = model.geometry.get_all_material_cells().values()
mgxs_lib_order5.build_library()
mgxs_lib_order5.check_library_for_openmc_mgxs()

/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 5 is greater than zero
  warn(msg, RuntimeWarning)


In [21]:
mgxs_lib_order7 = openmc.mgxs.Library(model.geometry)
groups = openmc.mgxs.EnergyGroups(egroup)

mgxs_lib_order7.energy_groups = groups
mgxs_lib_order7.scatter_format = "legendre"
mgxs_lib_order7.mgxs_types = ['transport','nu-transport','absorption', 'nu-fission', 'fission' ,\
                              'chi', 'consistent scatter matrix', 'multiplicity matrix','total',\
                              'consistent nu-scatter matrix']

mgxs_lib_order7.legendre_order = 7

mgxs_lib_order7.by_nuclide = False
mgxs_lib_order7.domain_type = 'cell'
mgxs_lib_order7.domains = model.geometry.get_all_material_cells().values()
mgxs_lib_order7.build_library()
mgxs_lib_order7.check_library_for_openmc_mgxs()

/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 7 is greater than zero
  warn(msg, RuntimeWarning)


In [22]:
tallies = openmc.Tallies()
mgxs_lib_order0.add_to_tallies_file(tallies, merge=False)
mgxs_lib_order1.add_to_tallies_file(tallies, merge=False)
mgxs_lib_order3.add_to_tallies_file(tallies, merge=False)
mgxs_lib_order5.add_to_tallies_file(tallies, merge=False)
mgxs_lib_order7.add_to_tallies_file(tallies, merge=False)

model.tallies = tallies

### Create power tally

model.export_to_model_xml('model_iso.xml')

### Run OpenMC model

In [23]:
statepoint_filename = model.run()

/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=1.
  warn(msg, IDWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=2.
  warn(msg, IDWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=3.
  warn(msg, IDWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=4.
  warn(msg, IDWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=21.
  warn(msg, IDWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=31.
  warn(msg, IDWarning)
/Users/jean.ragusa/anacond

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

       43/1    1.19154    1.18846 +/- 0.00214
 Triggers unsatisfied, max unc./thresh. is 4.883251973293909 for eigenvalue
 The estimated number of batches is 569
       44/1    1.19059    1.18855 +/- 0.00205
 Triggers unsatisfied, max unc./thresh. is 4.6961964169573625 for eigenvalue
 The estimated number of batches is 550
       45/1    1.17200    1.18789 +/- 0.00208
 Triggers unsatisfied, max unc./thresh. is 4.437113798981899 for eigenvalue
 The estimated number of batches is 513
       46/1    1.19117    1.18802 +/- 0.00200
 Triggers unsatisfied, max unc./thresh. is 4.2793552149206935 for eigenvalue
 The estimated number of batches is 497
       47/1    1.19021    1.18810 +/- 0.00193
 Triggers unsatisfied, max unc./thresh. is 4.179002142637941 for eigenvalue
 The estimated number of batches is 492
       48/1    1.18768    1.18808 +/- 0.00186
 Triggers unsatisfied, max unc./thresh. is 4.014138567989159 for eigenvalue
 The estimated number of batches is 472
       49/1    1.16901    

       94/1    1.16330    1.18539 +/- 0.00132
 Triggers unsatisfied, max unc./thresh. is 2.807311884409103 for eigenvalue
 The estimated number of batches is 604
       95/1    1.18356    1.18537 +/- 0.00131
 Triggers unsatisfied, max unc./thresh. is 2.774054482331357 for eigenvalue
 The estimated number of batches is 598
       96/1    1.18687    1.18539 +/- 0.00129
 Triggers unsatisfied, max unc./thresh. is 2.7610729271230983 for eigenvalue
 The estimated number of batches is 600
       97/1    1.16369    1.18511 +/- 0.00130
 Triggers unsatisfied, max unc./thresh. is 2.8128926116873085 for eigenvalue
 The estimated number of batches is 630
       98/1    1.19096    1.18518 +/- 0.00129
 Triggers unsatisfied, max unc./thresh. is 2.784865229402034 for eigenvalue
 The estimated number of batches is 625
       99/1    1.20209    1.18540 +/- 0.00129
 Triggers unsatisfied, max unc./thresh. is 2.7626094845975047 for eigenvalue
 The estimated number of batches is 623
      100/1    1.18019   

      145/1    1.19259    1.18583 +/- 0.00098
 Triggers unsatisfied, max unc./thresh. is 1.914429781037106 for eigenvalue
 The estimated number of batches is 479
      146/1    1.17604    1.18575 +/- 0.00097
 Triggers unsatisfied, max unc./thresh. is 1.910355153486107 for eigenvalue
 The estimated number of batches is 480
      147/1    1.18438    1.18574 +/- 0.00096
 Triggers unsatisfied, max unc./thresh. is 1.9006047609469008 for eigenvalue
 The estimated number of batches is 479
      148/1    1.19933    1.18585 +/- 0.00096
 Triggers unsatisfied, max unc./thresh. is 1.8915113449433254 for eigenvalue
 The estimated number of batches is 478
      149/1    1.17377    1.18575 +/- 0.00096
 Triggers unsatisfied, max unc./thresh. is 1.8846483134298715 for eigenvalue
 The estimated number of batches is 479
      150/1    1.20365    1.18589 +/- 0.00096
 Triggers unsatisfied, max unc./thresh. is 1.8748614429157417 for eigenvalue
 The estimated number of batches is 477
      151/1    1.18998  

...

      247/1    1.18463    1.18541 +/- 0.00073
 Triggers unsatisfied, max unc./thresh. is 1.346593587569123 for eigenvalue
 The estimated number of batches is 432
      248/1    1.18636    1.18541 +/- 0.00072
 Triggers unsatisfied, max unc./thresh. is 1.3458468803739652 for eigenvalue
 The estimated number of batches is 433
      249/1    1.18592    1.18541 +/- 0.00072
 Triggers unsatisfied, max unc./thresh. is 1.3439998016104313 for eigenvalue
 The estimated number of batches is 434
      250/1    1.17765    1.18538 +/- 0.00072
 Triggers unsatisfied, max unc./thresh. is 1.3414931863837565 for eigenvalue
 The estimated number of batches is 434
      251/1    1.18179    1.18536 +/- 0.00072
 Triggers unsatisfied, max unc./thresh. is 1.3366924835155474 for eigenvalue
 The estimated number of batches is 433
      252/1    1.17693    1.18533 +/- 0.00071
 Triggers unsatisfied, max unc./thresh. is 1.331277071643693 for eigenvalue
 The estimated number of batches is 432
      253/1    1.19146  

      298/1    1.20251    1.18547 +/- 0.00064
 Triggers unsatisfied, max unc./thresh. is 1.2276628841411348 for eigenvalue
 The estimated number of batches is 439
      299/1    1.19134    1.18549 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.2243871874538859 for eigenvalue
 The estimated number of batches is 439
      300/1    1.20036    1.18555 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.2196623839278622 for eigenvalue
 The estimated number of batches is 437
      301/1    1.19266    1.18557 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.215791029942085 for eigenvalue
 The estimated number of batches is 436
      302/1    1.18806    1.18558 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.2119575259095843 for eigenvalue
 The estimated number of batches is 435
      303/1    1.18239    1.18557 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.2077610231158804 for eigenvalue
 The estimated number of batches is 433
      304/1    1.20221 

      349/1    1.20106    1.18565 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.1557914265384148 for eigenvalue
 The estimated number of batches is 460
      350/1    1.18224    1.18564 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.152322999085731 for eigenvalue
 The estimated number of batches is 459
      351/1    1.19508    1.18567 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.1494568273055679 for eigenvalue
 The estimated number of batches is 458
      352/1    1.18897    1.18568 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.1469097747585666 for eigenvalue
 The estimated number of batches is 457
      353/1    1.20134    1.18573 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.1449815741078258 for eigenvalue
 The estimated number of batches is 457
      354/1    1.18982    1.18574 +/- 0.00058
 Triggers unsatisfied, max unc./thresh. is 1.1420985035927238 for eigenvalue
 The estimated number of batches is 456
      355/1    1.16650 

      400/1    1.19875    1.18598 +/- 0.00055
 Triggers unsatisfied, max unc./thresh. is 1.075512755527595 for eigenvalue
 The estimated number of batches is 460
      401/1    1.19151    1.18600 +/- 0.00055
 Triggers unsatisfied, max unc./thresh. is 1.0732042386182021 for eigenvalue
 The estimated number of batches is 459
      402/1    1.20354    1.18604 +/- 0.00055
 Triggers unsatisfied, max unc./thresh. is 1.0714451733062011 for eigenvalue
 The estimated number of batches is 459
      403/1    1.19059    1.18606 +/- 0.00055
 Triggers unsatisfied, max unc./thresh. is 1.0688177022576126 for eigenvalue
 The estimated number of batches is 458
      404/1    1.18731    1.18606 +/- 0.00055
 Triggers unsatisfied, max unc./thresh. is 1.0663359693669565 for eigenvalue
 The estimated number of batches is 457
      405/1    1.18999    1.18607 +/- 0.00054
 Triggers unsatisfied, max unc./thresh. is 1.064268073844177 for eigenvalue
 The estimated number of batches is 457
      406/1    1.19160  

      451/1    1.18702    1.18663 +/- 0.00052
 Triggers satisfied for batch 451
 Creating state point statepoint.00451.h5...

 =======================>     TIMING STATISTICS     <=======================

 Total time for initialization     = 4.3978e+00 seconds
   Reading cross sections          = 4.3816e+00 seconds
 Total time in simulation          = 7.7992e+02 seconds
   Time in transport only          = 7.6630e+02 seconds
   Time in inactive batches        = 2.6979e+00 seconds
   Time in active batches          = 7.7722e+02 seconds
   Time synchronizing fission bank = 2.8189e-01 seconds
     Sampling source sites         = 2.5299e-01 seconds
     SEND/RECV source sites        = 2.8716e-02 seconds
   Time accumulating tallies       = 1.2771e+01 seconds
   Time writing statepoints        = 4.5556e-01 seconds
 Total time for finalization       = 5.0298e+00 seconds
 Total time elapsed                = 7.8937e+02 seconds
 Calculation Rate (inactive)       = 148261 particles/second
 Calcul

In [24]:
ce_spfile = './statepoint_ce.h5'
os.rename(statepoint_filename, ce_spfile)

ce_sumfile = './summary_ce.h5'
os.rename('summary.h5', ce_sumfile)

In [25]:
sp = openmc.StatePoint(ce_spfile, autolink=False)
su = openmc.Summary(ce_sumfile)
sp.link_with_summary(su)

In [26]:
mgxs_lib_order0.load_from_statepoint(sp)
mgxs_lib_order1.load_from_statepoint(sp)
mgxs_lib_order3.load_from_statepoint(sp)
mgxs_lib_order5.load_from_statepoint(sp)
mgxs_lib_order7.load_from_statepoint(sp)

mgxs_file_order0 = mgxs_lib_order0.create_mg_library(xs_type='macro', xsdata_names=['fuel', 'clad', 'moderator'])
mgxs_file_order1 = mgxs_lib_order1.create_mg_library(xs_type='macro', xsdata_names=['fuel', 'clad', 'moderator'])
mgxs_file_order3 = mgxs_lib_order3.create_mg_library(xs_type='macro', xsdata_names=['fuel', 'clad', 'moderator'])
mgxs_file_order5 = mgxs_lib_order5.create_mg_library(xs_type='macro', xsdata_names=['fuel', 'clad', 'moderator'])
mgxs_file_order7 = mgxs_lib_order7.create_mg_library(xs_type='macro', xsdata_names=['fuel', 'clad', 'moderator'])

In [29]:
model_order0 = openmc.Model()
mgxs_file_order0, materials_file, geometry_file = mgxs_lib_order0.create_mg_mode()

mgxs_file_order0.xsdatas[0].temperatures=[565.]
mgxs_file_order0.xsdatas[1].temperatures=[565.]
mgxs_file_order0.xsdatas[2].temperatures=[565.]


mgxs_file_order0.export_to_hdf5('mgxs_order0.h5')
materials_file.cross_sections = 'mgxs_order0.h5'
model_order0.materials = materials_file
model_order0.geometry = geometry_file
settings_file.energy_mode = 'multi-group'
model_order0.settings = settings_file

In [30]:
statepoint_filename0 = model_order0.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

 Preparing distributed cell instances...
 Writing summary.h5 file...
 Initializing source particles...

 ====================>     K EIGENVALUE SIMULATION     <====================

  Bat./Gen.      k            Average k
  =========   ========   ====================
        1/1    1.16604
        2/1    1.19871
        3/1    1.19550


        4/1    1.16360
        5/1    1.18890
        6/1    1.19257
        7/1    1.18796
        8/1    1.18507
        9/1    1.18460
       10/1    1.18705
       11/1    1.18346
       12/1    1.17335
       13/1    1.16700
       14/1    1.18335
       15/1    1.17179
       16/1    1.18209
       17/1    1.18507
       18/1    1.17804
       19/1    1.16630
       20/1    1.17961
       21/1    1.18486
       22/1    1.18152    1.18319 +/- 0.00167
       23/1    1.18866    1.18501 +/- 0.00206
       24/1    1.18082    1.18397 +/- 0.00180
       25/1    1.17695    1.18256 +/- 0.00198
       26/1    1.18989    1.18378 +/- 0.00202
       27/1    1.18562    1.18405 +/- 0.00173
       28/1    1.17300    1.18266 +/- 0.00204
       29/1    1.16797    1.18103 +/- 0.00243
       30/1    1.16353    1.17928 +/- 0.00279
 Triggers unsatisfied, max unc./thresh. is 6.339186470613451 for eigenvalue
 The estimated number of batches is 422
 Creating state point statepoint.00030.h5...
       31/1

       76/1    1.20438    1.17969 +/- 0.00136
 Triggers unsatisfied, max unc./thresh. is 2.6265803012684694 for eigenvalue
 The estimated number of batches is 407
       77/1    1.17724    1.17964 +/- 0.00134
 Triggers unsatisfied, max unc./thresh. is 2.590288997834568 for eigenvalue
 The estimated number of batches is 403
       78/1    1.17371    1.17954 +/- 0.00132
 Triggers unsatisfied, max unc./thresh. is 2.5489401495038813 for eigenvalue
 The estimated number of batches is 397
       79/1    1.17906    1.17953 +/- 0.00130
 Triggers unsatisfied, max unc./thresh. is 2.5107741431606487 for eigenvalue
 The estimated number of batches is 392
       80/1    1.18834    1.17968 +/- 0.00129
 Triggers unsatisfied, max unc./thresh. is 2.4709978613935397 for eigenvalue
 The estimated number of batches is 387
       81/1    1.18732    1.17980 +/- 0.00127
 Triggers unsatisfied, max unc./thresh. is 2.484007322072316 for eigenvalue
 The estimated number of batches is 397
       82/1    1.18276  

      127/1    1.17066    1.18015 +/- 0.00098
 Triggers unsatisfied, max unc./thresh. is 1.8526012216363765 for eigenvalue
 The estimated number of batches is 388
      128/1    1.20364    1.18037 +/- 0.00099
 Triggers unsatisfied, max unc./thresh. is 1.8481937773135766 for eigenvalue
 The estimated number of batches is 389
      129/1    1.17490    1.18032 +/- 0.00099
 Triggers unsatisfied, max unc./thresh. is 1.832457522439086 for eigenvalue
 The estimated number of batches is 387
      130/1    1.18144    1.18033 +/- 0.00098
 Triggers unsatisfied, max unc./thresh. is 1.8294144928621445 for eigenvalue
 The estimated number of batches is 389
      131/1    1.18707    1.18039 +/- 0.00097
 Triggers unsatisfied, max unc./thresh. is 1.8129932990283564 for eigenvalue
 The estimated number of batches is 385
      132/1    1.18536    1.18043 +/- 0.00096
 Triggers unsatisfied, max unc./thresh. is 1.7968965598787716 for eigenvalue
 The estimated number of batches is 382
      133/1    1.19259 

      181/1    1.17031    1.18111 +/- 0.00079
 Triggers unsatisfied, max unc./thresh. is 1.5677560341894594 for eigenvalue
 The estimated number of batches is 416
      182/1    1.18278    1.18112 +/- 0.00078
 Triggers unsatisfied, max unc./thresh. is 1.5711319810457365 for eigenvalue
 The estimated number of batches is 420
      183/1    1.19615    1.18122 +/- 0.00078
 Triggers unsatisfied, max unc./thresh. is 1.5738865613747133 for eigenvalue
 The estimated number of batches is 424
      184/1    1.18394    1.18123 +/- 0.00078
 Triggers unsatisfied, max unc./thresh. is 1.5638741271892118 for eigenvalue
 The estimated number of batches is 422
      185/1    1.18520    1.18126 +/- 0.00077
 Triggers unsatisfied, max unc./thresh. is 1.5575787261729386 for eigenvalue
 The estimated number of batches is 421
      186/1    1.19328    1.18133 +/- 0.00077
 Triggers unsatisfied, max unc./thresh. is 1.5487401124210305 for eigenvalue
 The estimated number of batches is 419
      187/1    1.19206

      232/1    1.17928    1.18145 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.3099326999404655 for eigenvalue
 The estimated number of batches is 384
      233/1    1.17618    1.18142 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.3039912909342644 for eigenvalue
 The estimated number of batches is 383
      234/1    1.19381    1.18148 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.30955002501039 for eigenvalue
 The estimated number of batches is 387
      235/1    1.18808    1.18151 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.3056544759651196 for eigenvalue
 The estimated number of batches is 387
      236/1    1.18475    1.18153 +/- 0.00066
 Triggers unsatisfied, max unc./thresh. is 1.2996651096422094 for eigenvalue
 The estimated number of batches is 385
      237/1    1.18352    1.18154 +/- 0.00066
 Triggers unsatisfied, max unc./thresh. is 1.2999901425845264 for eigenvalue
 The estimated number of batches is 387
      238/1    1.17024  

      284/1    1.18533    1.18107 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.1545770089221952 for eigenvalue
 The estimated number of batches is 372
      285/1    1.17126    1.18104 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.1569845674749568 for eigenvalue
 The estimated number of batches is 375
      286/1    1.17668    1.18102 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.1585873637783553 for eigenvalue
 The estimated number of batches is 378
      287/1    1.18577    1.18104 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.1544847688061286 for eigenvalue
 The estimated number of batches is 376
      288/1    1.16978    1.18100 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.151859016000208 for eigenvalue
 The estimated number of batches is 376
      289/1    1.18804    1.18102 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.1476357753178728 for eigenvalue
 The estimated number of batches is 375
      290/1    1.17912 

      336/1    1.18262    1.18095 +/- 0.00054
 Triggers unsatisfied, max unc./thresh. is 1.0505870593214335 for eigenvalue
 The estimated number of batches is 369
      337/1    1.17958    1.18095 +/- 0.00054
 Triggers unsatisfied, max unc./thresh. is 1.0488418451414712 for eigenvalue
 The estimated number of batches is 369
      338/1    1.18729    1.18097 +/- 0.00054
 Triggers unsatisfied, max unc./thresh. is 1.0474933629847212 for eigenvalue
 The estimated number of batches is 369
      339/1    1.19102    1.18100 +/- 0.00054
 Triggers unsatisfied, max unc./thresh. is 1.0452702650857102 for eigenvalue
 The estimated number of batches is 369
      340/1    1.17678    1.18098 +/- 0.00054
 Triggers unsatisfied, max unc./thresh. is 1.0439476649874326 for eigenvalue
 The estimated number of batches is 369
      341/1    1.17852    1.18098 +/- 0.00054
 Triggers unsatisfied, max unc./thresh. is 1.040867147253748 for eigenvalue
 The estimated number of batches is 368
      342/1    1.18694 

In [31]:
model_order1 = openmc.Model()
mgxs_file_order1, materials_file, geometry_file = mgxs_lib_order1.create_mg_mode()

mgxs_file_order1.xsdatas[0].temperatures=[565.]
mgxs_file_order1.xsdatas[1].temperatures=[565.]
mgxs_file_order1.xsdatas[2].temperatures=[565.]
materials_file.cross_sections = 'mgxs_order1.h5'

mgxs_file_order1.export_to_hdf5('mgxs_order1.h5')
model_order1.materials = materials_file
model_order1.geometry = geometry_file
settings_file.energy_mode = 'multi-group'
model_order1.settings = settings_file

In [32]:
statepoint_filename1 = model_order1.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

 Preparing distributed cell instances...
 Writing summary.h5 file...
 Initializing source particles...

 ====================>     K EIGENVALUE SIMULATION     <====================

  Bat./Gen.      k            Average k
  =========   ========   ====================
        1/1    1.17525
        2/1    1.18265


        3/1    1.18723
        4/1    1.17670
        5/1    1.18274
        6/1    1.17891
        7/1    1.17762
        8/1    1.18327
        9/1    1.19012
       10/1    1.18714
       11/1    1.19438
       12/1    1.17744
       13/1    1.18386
       14/1    1.17929
       15/1    1.19893
       16/1    1.18286
       17/1    1.18243
       18/1    1.18576
       19/1    1.16519
       20/1    1.19078
       21/1    1.20289
       22/1    1.19594    1.19941 +/- 0.00347
       23/1    1.18830    1.19571 +/- 0.00421
       24/1    1.17958    1.19168 +/- 0.00501
       25/1    1.17536    1.18841 +/- 0.00507
       26/1    1.20472    1.19113 +/- 0.00495
       27/1    1.17930    1.18944 +/- 0.00452
       28/1    1.18690    1.18912 +/- 0.00392
       29/1    1.18855    1.18906 +/- 0.00346
       30/1    1.19537    1.18969 +/- 0.00316
 Triggers unsatisfied, max unc./thresh. is 6.428551677745427 for eigenvalue
 The estimated number of batches is 434
 Creating state point statepoint.

       77/1    1.15906    1.18042 +/- 0.00157
 Triggers unsatisfied, max unc./thresh. is 2.692482031290756 for eigenvalue
 The estimated number of batches is 434
       78/1    1.16375    1.18013 +/- 0.00157
 Triggers unsatisfied, max unc./thresh. is 2.654617994634492 for eigenvalue
 The estimated number of batches is 429
       79/1    1.19565    1.18039 +/- 0.00157
 Triggers unsatisfied, max unc./thresh. is 2.6359686479417057 for eigenvalue
 The estimated number of batches is 430
       80/1    1.16287    1.18010 +/- 0.00157
 Triggers unsatisfied, max unc./thresh. is 2.6564543408720644 for eigenvalue
 The estimated number of batches is 444
       81/1    1.18187    1.18013 +/- 0.00154
 Triggers unsatisfied, max unc./thresh. is 2.609798545305572 for eigenvalue
 The estimated number of batches is 436
       82/1    1.17334    1.18002 +/- 0.00152
 Triggers unsatisfied, max unc./thresh. is 2.570131208114095 for eigenvalue
 The estimated number of batches is 430
       83/1    1.17845    

      129/1    1.16843    1.18053 +/- 0.00113
 Triggers unsatisfied, max unc./thresh. is 1.969540793515605 for eigenvalue
 The estimated number of batches is 443
      130/1    1.16577    1.18039 +/- 0.00112
 Triggers unsatisfied, max unc./thresh. is 1.9571363830565303 for eigenvalue
 The estimated number of batches is 442
      131/1    1.17535    1.18035 +/- 0.00111
 Triggers unsatisfied, max unc./thresh. is 1.9431297249780126 for eigenvalue
 The estimated number of batches is 440
      132/1    1.17193    1.18027 +/- 0.00111
 Triggers unsatisfied, max unc./thresh. is 1.9256819995981314 for eigenvalue
 The estimated number of batches is 436
      133/1    1.19051    1.18036 +/- 0.00110
 Triggers unsatisfied, max unc./thresh. is 1.90888738998235 for eigenvalue
 The estimated number of batches is 432
      134/1    1.18188    1.18037 +/- 0.00109
 Triggers unsatisfied, max unc./thresh. is 1.8907504924863325 for eigenvalue
 The estimated number of batches is 428
      135/1    1.19065   

      180/1    1.16121    1.18133 +/- 0.00090
 Triggers unsatisfied, max unc./thresh. is 1.511319753825385 for eigenvalue
 The estimated number of batches is 386
      181/1    1.17113    1.18127 +/- 0.00089
 Triggers unsatisfied, max unc./thresh. is 1.513344435584414 for eigenvalue
 The estimated number of batches is 389
      182/1    1.18450    1.18129 +/- 0.00089
 Triggers unsatisfied, max unc./thresh. is 1.5045109641990262 for eigenvalue
 The estimated number of batches is 387
      183/1    1.15742    1.18114 +/- 0.00089
 Triggers unsatisfied, max unc./thresh. is 1.5099196616745476 for eigenvalue
 The estimated number of batches is 392
      184/1    1.19079    1.18120 +/- 0.00089
 Triggers unsatisfied, max unc./thresh. is 1.5024404090762566 for eigenvalue
 The estimated number of batches is 391
      185/1    1.17687    1.18117 +/- 0.00089
 Triggers unsatisfied, max unc./thresh. is 1.5004885917041961 for eigenvalue
 The estimated number of batches is 392
      186/1    1.18861  

      234/1    1.18805    1.18139 +/- 0.00076
 Triggers unsatisfied, max unc./thresh. is 1.303066638589456 for eigenvalue
 The estimated number of batches is 384
      235/1    1.17250    1.18135 +/- 0.00076
 Triggers unsatisfied, max unc./thresh. is 1.2979125421181448 for eigenvalue
 The estimated number of batches is 383
      236/1    1.20417    1.18145 +/- 0.00076
 Triggers unsatisfied, max unc./thresh. is 1.2925520760192413 for eigenvalue
 The estimated number of batches is 381
      237/1    1.19738    1.18153 +/- 0.00076
 Triggers unsatisfied, max unc./thresh. is 1.2913015939063206 for eigenvalue
 The estimated number of batches is 382
      238/1    1.18977    1.18156 +/- 0.00076
 Triggers unsatisfied, max unc./thresh. is 1.288781881505156 for eigenvalue
 The estimated number of batches is 383
      239/1    1.18658    1.18159 +/- 0.00075
 Triggers unsatisfied, max unc./thresh. is 1.290231278862824 for eigenvalue
 The estimated number of batches is 385
      240/1    1.19500   

      287/1    1.18717    1.18185 +/- 0.00068
 Triggers unsatisfied, max unc./thresh. is 1.1911722289941082 for eigenvalue
 The estimated number of batches is 399
      288/1    1.16712    1.18179 +/- 0.00068
 Triggers unsatisfied, max unc./thresh. is 1.1864556174997574 for eigenvalue
 The estimated number of batches is 398
      289/1    1.17237    1.18176 +/- 0.00068
 Triggers unsatisfied, max unc./thresh. is 1.181687482469033 for eigenvalue
 The estimated number of batches is 396
      290/1    1.19849    1.18182 +/- 0.00068
 Triggers unsatisfied, max unc./thresh. is 1.18111547387798 for eigenvalue
 The estimated number of batches is 397
      291/1    1.18246    1.18182 +/- 0.00068
 Triggers unsatisfied, max unc./thresh. is 1.1791931729325822 for eigenvalue
 The estimated number of batches is 397
      292/1    1.17218    1.18179 +/- 0.00068
 Triggers unsatisfied, max unc./thresh. is 1.1785155260388247 for eigenvalue
 The estimated number of batches is 398
      293/1    1.18584   

      339/1    1.16494    1.18142 +/- 0.00061
 Triggers unsatisfied, max unc./thresh. is 1.069862111689583 for eigenvalue
 The estimated number of batches is 386
      340/1    1.17962    1.18141 +/- 0.00061
 Triggers unsatisfied, max unc./thresh. is 1.066612607500971 for eigenvalue
 The estimated number of batches is 385
      341/1    1.17248    1.18139 +/- 0.00061
 Triggers unsatisfied, max unc./thresh. is 1.0634124618351455 for eigenvalue
 The estimated number of batches is 384
      342/1    1.17749    1.18137 +/- 0.00061
 Triggers unsatisfied, max unc./thresh. is 1.0608028512769068 for eigenvalue
 The estimated number of batches is 383
      343/1    1.18605    1.18139 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.0576364233371223 for eigenvalue
 The estimated number of batches is 382
      344/1    1.17018    1.18135 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.0579321204326038 for eigenvalue
 The estimated number of batches is 383
      345/1    1.18662  

      390/1    1.20103    1.18154 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0208427498757724 for eigenvalue
 The estimated number of batches is 406
      391/1    1.19174    1.18157 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0182453559255267 for eigenvalue
 The estimated number of batches is 405
      392/1    1.17371    1.18154 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0152638113642927 for eigenvalue
 The estimated number of batches is 404
      393/1    1.17579    1.18153 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.015741623470997 for eigenvalue
 The estimated number of batches is 405
      394/1    1.16375    1.18148 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0135875490823354 for eigenvalue
 The estimated number of batches is 405
      395/1    1.17640    1.18147 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0124328281482875 for eigenvalue
 The estimated number of batches is 405
      396/1    1.19332 

In [33]:
model_order3 = openmc.Model()
mgxs_file_order3, materials_file, geometry_file = mgxs_lib_order3.create_mg_mode()

mgxs_file_order3.xsdatas[0].temperatures=[565.]
mgxs_file_order3.xsdatas[1].temperatures=[565.]
mgxs_file_order3.xsdatas[2].temperatures=[565.]
materials_file.cross_sections = 'mgxs_order3.h5'

mgxs_file_order3.export_to_hdf5('mgxs_order3.h5')
model_order3.materials = materials_file
model_order3.geometry = geometry_file
settings_file.energy_mode = 'multi-group'
model_order3.settings = settings_file

In [34]:
statepoint_filename3 = model_order3.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

 Preparing distributed cell instances...
 Writing summary.h5 file...
 Initializing source particles...

 ====================>     K EIGENVALUE SIMULATION     <====================

  Bat./Gen.      k            Average k
  =========   ========   ====================
        1/1    1.19796
        2/1    1.18373


        3/1    1.18898
        4/1    1.18229
        5/1    1.16969
        6/1    1.19211
        7/1    1.17906
        8/1    1.19208
        9/1    1.18824
       10/1    1.17119
       11/1    1.17024
       12/1    1.16896
       13/1    1.20324
       14/1    1.18902
       15/1    1.19341
       16/1    1.17575
       17/1    1.18769
       18/1    1.18848
       19/1    1.18733
       20/1    1.19021
       21/1    1.17479
       22/1    1.19316    1.18398 +/- 0.00918
       23/1    1.15831    1.17542 +/- 0.01006
       24/1    1.17700    1.17582 +/- 0.00713
       25/1    1.17789    1.17623 +/- 0.00554
       26/1    1.18100    1.17703 +/- 0.00459
       27/1    1.19908    1.18018 +/- 0.00500
       28/1    1.19994    1.18265 +/- 0.00498
       29/1    1.19734    1.18428 +/- 0.00469
       30/1    1.17886    1.18374 +/- 0.00423
 Triggers unsatisfied, max unc./thresh. is 9.578381285997832 for eigenvalue
 The estimated number of batches is 938
 Creating state point statepoint.

       77/1    1.17536    1.18365 +/- 0.00133
 Triggers unsatisfied, max unc./thresh. is 2.3859871580980605 for eigenvalue
 The estimated number of batches is 345
       78/1    1.18373    1.18365 +/- 0.00131
 Triggers unsatisfied, max unc./thresh. is 2.344231773697762 for eigenvalue
 The estimated number of batches is 339
       79/1    1.16908    1.18340 +/- 0.00131
 Triggers unsatisfied, max unc./thresh. is 2.349842248448706 for eigenvalue
 The estimated number of batches is 346
       80/1    1.17130    1.18320 +/- 0.00130
 Triggers unsatisfied, max unc./thresh. is 2.311684877777045 for eigenvalue
 The estimated number of batches is 341
       81/1    1.18283    1.18320 +/- 0.00128
 Triggers unsatisfied, max unc./thresh. is 2.277395040984342 for eigenvalue
 The estimated number of batches is 337
       82/1    1.16771    1.18295 +/- 0.00129
 Triggers unsatisfied, max unc./thresh. is 2.239050264902833 for eigenvalue
 The estimated number of batches is 331
       83/1    1.18469    1

      129/1    1.16212    1.18259 +/- 0.00094
 Triggers unsatisfied, max unc./thresh. is 1.6626014000558111 for eigenvalue
 The estimated number of batches is 322
      130/1    1.18143    1.18257 +/- 0.00093
 Triggers unsatisfied, max unc./thresh. is 1.6585769441364395 for eigenvalue
 The estimated number of batches is 323
      131/1    1.17158    1.18248 +/- 0.00093
 Triggers unsatisfied, max unc./thresh. is 1.6438875611663128 for eigenvalue
 The estimated number of batches is 320
      132/1    1.18538    1.18250 +/- 0.00092
 Triggers unsatisfied, max unc./thresh. is 1.6289099185307765 for eigenvalue
 The estimated number of batches is 318
      133/1    1.17186    1.18241 +/- 0.00092
 Triggers unsatisfied, max unc./thresh. is 1.6299622535615252 for eigenvalue
 The estimated number of batches is 321
      134/1    1.17462    1.18234 +/- 0.00091
 Triggers unsatisfied, max unc./thresh. is 1.6364871293331182 for eigenvalue
 The estimated number of batches is 326
      135/1    1.19344

      181/1    1.18654    1.18222 +/- 0.00076
 Triggers unsatisfied, max unc./thresh. is 1.427224228623223 for eigenvalue
 The estimated number of batches is 348
      182/1    1.19230    1.18228 +/- 0.00076
 Triggers unsatisfied, max unc./thresh. is 1.418811929684475 for eigenvalue
 The estimated number of batches is 347
      183/1    1.17244    1.18222 +/- 0.00076
 Triggers unsatisfied, max unc./thresh. is 1.4104178321137044 for eigenvalue
 The estimated number of batches is 345
      184/1    1.17173    1.18215 +/- 0.00076
 Triggers unsatisfied, max unc./thresh. is 1.4045902157544152 for eigenvalue
 The estimated number of batches is 344
      185/1    1.18191    1.18215 +/- 0.00075
 Triggers unsatisfied, max unc./thresh. is 1.407070014382023 for eigenvalue
 The estimated number of batches is 347
      186/1    1.18053    1.18214 +/- 0.00075
 Triggers unsatisfied, max unc./thresh. is 1.3986764487877414 for eigenvalue
 The estimated number of batches is 345
      187/1    1.18166   

      235/1    1.17630    1.18121 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.223255988433554 for eigenvalue
 The estimated number of batches is 342
      236/1    1.17376    1.18118 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.2218662254942803 for eigenvalue
 The estimated number of batches is 343
      237/1    1.17298    1.18114 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.2170158338423744 for eigenvalue
 The estimated number of batches is 342
      238/1    1.16876    1.18109 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.2117986118016095 for eigenvalue
 The estimated number of batches is 341
      239/1    1.17994    1.18108 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.206662954234313 for eigenvalue
 The estimated number of batches is 339
      240/1    1.17901    1.18107 +/- 0.00066
 Triggers unsatisfied, max unc./thresh. is 1.2014849232676204 for eigenvalue
 The estimated number of batches is 338
      241/1    1.17224  

      289/1    1.18794    1.18113 +/- 0.00062
 Triggers unsatisfied, max unc./thresh. is 1.1109436457994526 for eigenvalue
 The estimated number of batches is 352
      290/1    1.17371    1.18110 +/- 0.00062
 Triggers unsatisfied, max unc./thresh. is 1.1074006183057339 for eigenvalue
 The estimated number of batches is 352
      291/1    1.19028    1.18113 +/- 0.00062
 Triggers unsatisfied, max unc./thresh. is 1.103146226743458 for eigenvalue
 The estimated number of batches is 350
      292/1    1.16684    1.18108 +/- 0.00062
 Triggers unsatisfied, max unc./thresh. is 1.110187491779036 for eigenvalue
 The estimated number of batches is 356
      293/1    1.20012    1.18115 +/- 0.00062
 Triggers unsatisfied, max unc./thresh. is 1.114591893729205 for eigenvalue
 The estimated number of batches is 360
      294/1    1.19786    1.18121 +/- 0.00062
 Triggers unsatisfied, max unc./thresh. is 1.1160557926915708 for eigenvalue
 The estimated number of batches is 362
      295/1    1.17472   

      340/1    1.16389    1.18110 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.0686528154989285 for eigenvalue
 The estimated number of batches is 386
      341/1    1.18240    1.18111 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.0658437432109034 for eigenvalue
 The estimated number of batches is 385
      342/1    1.17818    1.18110 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.062788224232241 for eigenvalue
 The estimated number of batches is 384
      343/1    1.18607    1.18111 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.0598302198183491 for eigenvalue
 The estimated number of batches is 383
      344/1    1.17662    1.18110 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.0573234517053354 for eigenvalue
 The estimated number of batches is 383
      345/1    1.18930    1.18112 +/- 0.00058
 Triggers unsatisfied, max unc./thresh. is 1.0542222801556156 for eigenvalue
 The estimated number of batches is 382
      346/1    1.19436 

In [35]:
model_order5 = openmc.Model()
mgxs_file_order5, materials_file, geometry_file = mgxs_lib_order5.create_mg_mode()

mgxs_file_order5.xsdatas[0].temperatures=[565.]
mgxs_file_order5.xsdatas[1].temperatures=[565.]
mgxs_file_order5.xsdatas[2].temperatures=[565.]
materials_file.cross_sections = 'mgxs_order5.h5'

mgxs_file_order5.export_to_hdf5('mgxs_order5.h5')
model_order5.materials = materials_file
model_order5.geometry = geometry_file
settings_file.energy_mode = 'multi-group'
model_order5.settings = settings_file

In [36]:
statepoint_filename5 = model_order5.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

 Preparing distributed cell instances...
 Writing summary.h5 file...
 Initializing source particles...

 ====================>     K EIGENVALUE SIMULATION     <====================

  Bat./Gen.      k            Average k
  =========   ========   ====================
        1/1    1.17261
        2/1    1.17814


        3/1    1.17556
        4/1    1.19151
        5/1    1.18343
        6/1    1.17479
        7/1    1.19258
        8/1    1.17565
        9/1    1.18398
       10/1    1.17048
       11/1    1.17578
       12/1    1.18246
       13/1    1.19874
       14/1    1.16570
       15/1    1.21031
       16/1    1.16239
       17/1    1.18217
       18/1    1.17125
       19/1    1.17824
       20/1    1.20323
       21/1    1.17215
       22/1    1.18219    1.17717 +/- 0.00502
       23/1    1.17647    1.17693 +/- 0.00291
       24/1    1.17873    1.17738 +/- 0.00210
       25/1    1.17222    1.17635 +/- 0.00193
       26/1    1.18326    1.17750 +/- 0.00195
       27/1    1.19414    1.17988 +/- 0.00289
       28/1    1.20913    1.18354 +/- 0.00443
       29/1    1.17557    1.18265 +/- 0.00401
       30/1    1.18342    1.18273 +/- 0.00359
 Triggers unsatisfied, max unc./thresh. is 8.057119535958545 for eigenvalue
 The estimated number of batches is 670
 Creating state point statepoint.

       78/1    1.18565    1.18090 +/- 0.00125
 Triggers unsatisfied, max unc./thresh. is 2.417764731967186 for eigenvalue
 The estimated number of batches is 360
       79/1    1.17567    1.18081 +/- 0.00123
 Triggers unsatisfied, max unc./thresh. is 2.3943538254615717 for eigenvalue
 The estimated number of batches is 359
       80/1    1.17194    1.18066 +/- 0.00122
 Triggers unsatisfied, max unc./thresh. is 2.3500376491855137 for eigenvalue
 The estimated number of batches is 352
       81/1    1.18535    1.18074 +/- 0.00120
 Triggers unsatisfied, max unc./thresh. is 2.387150183515952 for eigenvalue
 The estimated number of batches is 368
       82/1    1.18110    1.18074 +/- 0.00118
 Triggers unsatisfied, max unc./thresh. is 2.3465491309381736 for eigenvalue
 The estimated number of batches is 362
       83/1    1.17671    1.18068 +/- 0.00117
 Triggers unsatisfied, max unc./thresh. is 2.3104261791540566 for eigenvalue
 The estimated number of batches is 357
       84/1    1.18241  

      129/1    1.17215    1.17887 +/- 0.00092
 Triggers unsatisfied, max unc./thresh. is 1.8033805650673693 for eigenvalue
 The estimated number of batches is 375
      130/1    1.18805    1.17895 +/- 0.00092
 Triggers unsatisfied, max unc./thresh. is 1.7824291747227248 for eigenvalue
 The estimated number of batches is 370
      131/1    1.17834    1.17894 +/- 0.00091
 Triggers unsatisfied, max unc./thresh. is 1.7912515873466663 for eigenvalue
 The estimated number of batches is 377
      132/1    1.20197    1.17915 +/- 0.00093
 Triggers unsatisfied, max unc./thresh. is 1.8000752372812694 for eigenvalue
 The estimated number of batches is 383
      133/1    1.16938    1.17906 +/- 0.00092
 Triggers unsatisfied, max unc./thresh. is 1.7868172837228706 for eigenvalue
 The estimated number of batches is 381
      134/1    1.17821    1.17906 +/- 0.00091
 Triggers unsatisfied, max unc./thresh. is 1.791562507043483 for eigenvalue
 The estimated number of batches is 386
      135/1    1.17366 

      182/1    1.18153    1.18079 +/- 0.00083
 Triggers unsatisfied, max unc./thresh. is 1.5662625986330838 for eigenvalue
 The estimated number of batches is 418
      183/1    1.19094    1.18085 +/- 0.00083
 Triggers unsatisfied, max unc./thresh. is 1.5604450494878284 for eigenvalue
 The estimated number of batches is 417
      184/1    1.17530    1.18082 +/- 0.00082
 Triggers unsatisfied, max unc./thresh. is 1.5523447715312262 for eigenvalue
 The estimated number of batches is 416
      185/1    1.16903    1.18075 +/- 0.00082
 Triggers unsatisfied, max unc./thresh. is 1.5462412117605753 for eigenvalue
 The estimated number of batches is 415
      186/1    1.17356    1.18070 +/- 0.00082
 Triggers unsatisfied, max unc./thresh. is 1.5411554211920304 for eigenvalue
 The estimated number of batches is 415
      187/1    1.17231    1.18065 +/- 0.00081
 Triggers unsatisfied, max unc./thresh. is 1.5383331159345945 for eigenvalue
 The estimated number of batches is 416
      188/1    1.19786

      235/1    1.16816    1.18106 +/- 0.00073
 Triggers unsatisfied, max unc./thresh. is 1.3891708026262959 for eigenvalue
 The estimated number of batches is 435
      236/1    1.17749    1.18105 +/- 0.00073
 Triggers unsatisfied, max unc./thresh. is 1.3833315231350964 for eigenvalue
 The estimated number of batches is 434
      237/1    1.17460    1.18102 +/- 0.00073
 Triggers unsatisfied, max unc./thresh. is 1.377864416094286 for eigenvalue
 The estimated number of batches is 432
      238/1    1.18537    1.18104 +/- 0.00072
 Triggers unsatisfied, max unc./thresh. is 1.373048923672899 for eigenvalue
 The estimated number of batches is 431
      239/1    1.16847    1.18098 +/- 0.00072
 Triggers unsatisfied, max unc./thresh. is 1.3663813368584488 for eigenvalue
 The estimated number of batches is 429
      240/1    1.17192    1.18094 +/- 0.00072
 Triggers unsatisfied, max unc./thresh. is 1.3626904299065705 for eigenvalue
 The estimated number of batches is 429
      241/1    1.16456  

      286/1    1.19694    1.18101 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.2025717757826646 for eigenvalue
 The estimated number of batches is 405
      287/1    1.19390    1.18106 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.1975778134631196 for eigenvalue
 The estimated number of batches is 403
      288/1    1.17131    1.18102 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.1928311377888887 for eigenvalue
 The estimated number of batches is 402
      289/1    1.19997    1.18110 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.1891889139028764 for eigenvalue
 The estimated number of batches is 401
      290/1    1.17192    1.18106 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.1849061827025613 for eigenvalue
 The estimated number of batches is 400
      291/1    1.17516    1.18104 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.18220027631175 for eigenvalue
 The estimated number of batches is 399
      292/1    1.16766  

      340/1    1.17242    1.18110 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0645066195283486 for eigenvalue
 The estimated number of batches is 383
      341/1    1.17904    1.18109 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0620141160758938 for eigenvalue
 The estimated number of batches is 383
      342/1    1.17562    1.18108 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0587340289369778 for eigenvalue
 The estimated number of batches is 381
      343/1    1.18480    1.18109 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0558795343370497 for eigenvalue
 The estimated number of batches is 381
      344/1    1.17288    1.18106 +/- 0.00055
 Triggers unsatisfied, max unc./thresh. is 1.0527021725885484 for eigenvalue
 The estimated number of batches is 380
      345/1    1.16031    1.18100 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0530436584050473 for eigenvalue
 The estimated number of batches is 381
      346/1    1.17545

In [37]:
model_order7 = openmc.Model()
mgxs_file_order7, materials_file, geometry_file = mgxs_lib_order7.create_mg_mode()

mgxs_file_order7.xsdatas[0].temperatures=[565.]
mgxs_file_order7.xsdatas[1].temperatures=[565.]
mgxs_file_order7.xsdatas[2].temperatures=[565.]
materials_file.cross_sections = 'mgxs_order7.h5'

mgxs_file_order7.export_to_hdf5('mgxs_order7.h5')
model_order7.materials = materials_file
model_order7.geometry = geometry_file
settings_file.energy_mode = 'multi-group'
model_order7.settings = settings_file

In [38]:
statepoint_filename7 = model_order7.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

 Preparing distributed cell instances...
 Writing summary.h5 file...
 Initializing source particles...

 ====================>     K EIGENVALUE SIMULATION     <====================

  Bat./Gen.      k            Average k
  =========   ========   ====================
        1/1    1.18786
        2/1    1.19162


        3/1    1.18260
        4/1    1.19234
        5/1    1.17012
        6/1    1.17286
        7/1    1.17986
        8/1    1.17853
        9/1    1.17903
       10/1    1.17875
       11/1    1.16787
       12/1    1.16081
       13/1    1.17582
       14/1    1.19131
       15/1    1.17260
       16/1    1.17725
       17/1    1.16280
       18/1    1.18196
       19/1    1.16848
       20/1    1.19173
       21/1    1.17951
       22/1    1.20424    1.19188 +/- 0.01236
       23/1    1.19863    1.19413 +/- 0.00748
       24/1    1.18852    1.19273 +/- 0.00548
       25/1    1.15922    1.18602 +/- 0.00793
       26/1    1.18184    1.18533 +/- 0.00651
       27/1    1.19309    1.18644 +/- 0.00561
       28/1    1.17043    1.18444 +/- 0.00526
       29/1    1.20807    1.18706 +/- 0.00533
       30/1    1.20406    1.18876 +/- 0.00506
 Triggers unsatisfied, max unc./thresh. is 6.966414515661659 for eigenvalue
 The estimated number of batches is 506
 Creating state point statepoint.

       76/1    1.19600    1.18298 +/- 0.00178
 Triggers unsatisfied, max unc./thresh. is 2.8861227532353664 for eigenvalue
 The estimated number of batches is 487
       77/1    1.16293    1.18263 +/- 0.00179
 Triggers unsatisfied, max unc./thresh. is 2.9047145343927956 for eigenvalue
 The estimated number of batches is 501
       78/1    1.18277    1.18263 +/- 0.00176
 Triggers unsatisfied, max unc./thresh. is 2.9017844159457926 for eigenvalue
 The estimated number of batches is 509
       79/1    1.16993    1.18242 +/- 0.00174
 Triggers unsatisfied, max unc./thresh. is 2.8526032376214125 for eigenvalue
 The estimated number of batches is 501
       80/1    1.16152    1.18207 +/- 0.00175
 Triggers unsatisfied, max unc./thresh. is 2.849378433023929 for eigenvalue
 The estimated number of batches is 508
       81/1    1.17650    1.18198 +/- 0.00172
 Triggers unsatisfied, max unc./thresh. is 2.8461486480731315 for eigenvalue
 The estimated number of batches is 515
       82/1    1.17891 

      128/1    1.17049    1.18174 +/- 0.00118
 Triggers unsatisfied, max unc./thresh. is 2.1051674919216534 for eigenvalue
 The estimated number of batches is 499
      129/1    1.17815    1.18171 +/- 0.00117
 Triggers unsatisfied, max unc./thresh. is 2.089242694317829 for eigenvalue
 The estimated number of batches is 496
      130/1    1.17225    1.18162 +/- 0.00116
 Triggers unsatisfied, max unc./thresh. is 2.0684066721827277 for eigenvalue
 The estimated number of batches is 491
      131/1    1.18336    1.18164 +/- 0.00115
 Triggers unsatisfied, max unc./thresh. is 2.0505022716849637 for eigenvalue
 The estimated number of batches is 487
      132/1    1.18169    1.18164 +/- 0.00114
 Triggers unsatisfied, max unc./thresh. is 2.0362518606337816 for eigenvalue
 The estimated number of batches is 485
      133/1    1.18788    1.18169 +/- 0.00114
 Triggers unsatisfied, max unc./thresh. is 2.0486711374164215 for eigenvalue
 The estimated number of batches is 495
      134/1    1.17510 

      181/1    1.17645    1.18188 +/- 0.00089
 Triggers unsatisfied, max unc./thresh. is 1.67749408350848 for eigenvalue
 The estimated number of batches is 474
      182/1    1.19039    1.18193 +/- 0.00089
 Triggers unsatisfied, max unc./thresh. is 1.6680386579055169 for eigenvalue
 The estimated number of batches is 471
      183/1    1.16910    1.18185 +/- 0.00089
 Triggers unsatisfied, max unc./thresh. is 1.6603726502197478 for eigenvalue
 The estimated number of batches is 470
      184/1    1.17650    1.18182 +/- 0.00088
 Triggers unsatisfied, max unc./thresh. is 1.6530008080814302 for eigenvalue
 The estimated number of batches is 469
      185/1    1.16948    1.18174 +/- 0.00088
 Triggers unsatisfied, max unc./thresh. is 1.6692089575039855 for eigenvalue
 The estimated number of batches is 480
      186/1    1.18624    1.18177 +/- 0.00088
 Triggers unsatisfied, max unc./thresh. is 1.6610896341509374 for eigenvalue
 The estimated number of batches is 479
      187/1    1.19605  

      233/1    1.20209    1.18200 +/- 0.00077
 Triggers unsatisfied, max unc./thresh. is 1.384177850218606 for eigenvalue
 The estimated number of batches is 429
      234/1    1.17178    1.18195 +/- 0.00077
 Triggers unsatisfied, max unc./thresh. is 1.3791182602046657 for eigenvalue
 The estimated number of batches is 428
      235/1    1.18615    1.18197 +/- 0.00076
 Triggers unsatisfied, max unc./thresh. is 1.3788723092475235 for eigenvalue
 The estimated number of batches is 429
      236/1    1.16653    1.18190 +/- 0.00076
 Triggers unsatisfied, max unc./thresh. is 1.3723689915834296 for eigenvalue
 The estimated number of batches is 427
      237/1    1.16437    1.18182 +/- 0.00076
 Triggers unsatisfied, max unc./thresh. is 1.366171988946454 for eigenvalue
 The estimated number of batches is 426
      238/1    1.18101    1.18182 +/- 0.00076
 Triggers unsatisfied, max unc./thresh. is 1.3614065009480565 for eigenvalue
 The estimated number of batches is 425
      239/1    1.17804  

      285/1    1.17488    1.18204 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.2411629376891216 for eigenvalue
 The estimated number of batches is 429
      286/1    1.17791    1.18202 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.236487414621747 for eigenvalue
 The estimated number of batches is 427
      287/1    1.17605    1.18200 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.232063674429996 for eigenvalue
 The estimated number of batches is 426
      288/1    1.20205    1.18207 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.2306692148222365 for eigenvalue
 The estimated number of batches is 426
      289/1    1.18777    1.18210 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.2287626195154482 for eigenvalue
 The estimated number of batches is 427
      290/1    1.18306    1.18210 +/- 0.00066
 Triggers unsatisfied, max unc./thresh. is 1.226215699702018 for eigenvalue
 The estimated number of batches is 426
      291/1    1.17283   

      337/1    1.19274    1.18199 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.102214224777173 for eigenvalue
 The estimated number of batches is 406
      338/1    1.18555    1.18200 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.0990058140319183 for eigenvalue
 The estimated number of batches is 405
      339/1    1.17371    1.18198 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.0955789462490886 for eigenvalue
 The estimated number of batches is 403
      340/1    1.19848    1.18203 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.0937328736556347 for eigenvalue
 The estimated number of batches is 403
      341/1    1.17442    1.18201 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.1022088806372845 for eigenvalue
 The estimated number of batches is 410
      342/1    1.15704    1.18193 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.0994402546902324 for eigenvalue
 The estimated number of batches is 410
      343/1    1.18179 

      389/1    1.17650    1.18200 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0295708627530331 for eigenvalue
 The estimated number of batches is 412
      390/1    1.20832    1.18207 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0286923986884102 for eigenvalue
 The estimated number of batches is 412
      391/1    1.15853    1.18200 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0286484153466176 for eigenvalue
 The estimated number of batches is 413
      392/1    1.16702    1.18196 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0291609292304436 for eigenvalue
 The estimated number of batches is 415
      393/1    1.17119    1.18193 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0292203420906294 for eigenvalue
 The estimated number of batches is 416
      394/1    1.17405    1.18191 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.026291224309057 for eigenvalue
 The estimated number of batches is 414
      395/1    1.17906 